In [99]:
import os
import cv2
from keras.preprocessing import image
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [100]:
# Dataset path
dataset_path = r"F:/Cancer Pred/Blood cell Cancer [ALL]"

In [101]:
# Define paths for each class label
class_paths = {
    "Early Pre-B": r"F:\Cancer Pred\Blood cell Cancer [ALL]\[Malignant] Early Pre-B",
    "Pre-B": r"F:\Cancer Pred\Blood cell Cancer [ALL]\[Malignant] Pre-B",
    "Pro-B": r"F:\Cancer Pred\Blood cell Cancer [ALL]\[Malignant] Pro-B",
    "Benign": r"F:\Cancer Pred\Blood cell Cancer [ALL]\Benign"
}

# Check if directories exist
for class_label, class_path in class_paths.items():
    if not os.path.exists(class_path):
        print(f"Directory not found: {class_path}")

In [102]:
# Initialize lists to store images and labels
X = []
y = []


In [103]:
from keras.preprocessing.image import img_to_array, load_img
import numpy as np

# Function to load and preprocess images from a directory
def load_images_from_directory(directory, target_size=(128, 128)):
    images = []
    labels = []
    for image_name in os.listdir(directory):
        image_path = os.path.join(directory, image_name)
        try:
            # Load and preprocess the image
            image = load_img(image_path, target_size=target_size)
            image = img_to_array(image) / 255.0  # Normalize pixel values to [0, 1]
            images.append(image)
            labels.append(class_label)
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
    return np.array(images), np.array(labels)


In [104]:
# Initialize empty lists to store images and labels
all_images = []
all_labels = []

In [105]:
# Load images and labels from each class directory
for class_label, class_path in class_paths.items():
    if os.path.exists(class_path):
        images, labels = load_images_from_directory(class_path)
        all_images.extend(images)
        all_labels.extend(labels)
    else:
        print(f"Directory not found: {class_path}")


In [106]:
# Convert lists to numpy arrays
all_images = np.array(all_images)
all_labels = np.array(all_labels)


In [107]:
# Shuffle the data
indices = np.arange(all_images.shape[0])
np.random.shuffle(indices)
all_images = all_images[indices]
all_labels = all_labels[indices]

In [108]:
# Print the shape of the loaded data
print("Shape of images array:", all_images.shape)
print("Shape of labels array:", all_labels.shape)

Shape of images array: (3242, 128, 128, 3)
Shape of labels array: (3242,)


In [113]:
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=80)

# Resize the input images to match the model's input shape
X_train_resized = np.array([cv2.resize(image, (224, 224)) for image in X_train])
X_test_resized = np.array([cv2.resize(image, (224, 224)) for image in X_test])

In [117]:
from sklearn.preprocessing import LabelEncoder

# Convert string labels to numerical categories
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(class_labels), activation='softmax')
])


In [118]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [119]:
# Train the model
model.fit(X_train_resized, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test_resized, y_test_encoded))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_resized, y_test_encoded)
print(f"Test Accuracy: {test_acc}")


Epoch 1/10
82/82 [==============================] - 46s 524ms/step - loss: 0.9155 - accuracy: 0.6437 - val_loss: 0.5210 - val_accuracy: 0.7951
Epoch 2/10
82/82 [==============================] - 36s 442ms/step - loss: 0.3863 - accuracy: 0.8457 - val_loss: 0.3805 - val_accuracy: 0.8351
Epoch 3/10
82/82 [==============================] - 36s 443ms/step - loss: 0.3745 - accuracy: 0.8585 - val_loss: 0.7211 - val_accuracy: 0.6949
Epoch 4/10
82/82 [==============================] - 36s 442ms/step - loss: 0.2698 - accuracy: 0.8970 - val_loss: 0.4744 - val_accuracy: 0.8259
Epoch 5/10
82/82 [==============================] - 36s 440ms/step - loss: 0.2126 - accuracy: 0.9179 - val_loss: 0.3501 - val_accuracy: 0.8567
Epoch 6/10
82/82 [==============================] - 36s 441ms/step - loss: 0.1836 - accuracy: 0.9337 - val_loss: 0.3391 - val_accuracy: 0.8921
Epoch 7/10
82/82 [==============================] - 36s 440ms/step - loss: 0.1252 - accuracy: 0.9603 - val_loss: 0.3625 - val_accuracy: 0.8875

In [124]:
# Resize test images to match the input shape of the model
X_test_resized = np.array([cv2.resize(img, (224, 224)) for img in X_test])

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_resized, y_test_encoded)
print(f"Test Accuracy: {test_acc}")


21/21 [==============================] - 3s 131ms/step - loss: 0.5653 - accuracy: 0.8706
Test Accuracy: 0.8705701231956482


In [125]:
model.save("blood_cell1_cancer_model.h5")

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
